### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import pandas as pd

In [132]:
## Import Rent Data
# Original data source: https://www.kaggle.com/datasets/austinreese/usa-housing-listings
# Last updated Jan 2020
# Filtered down for 12 largest metros according to: https://worldpopulationreview.com/us-cities

# Read file and display
rent = "Resources/housing_majors.csv"
rent = pd.read_csv(rent)

# Standardize city names
cities_list = rent.drop_duplicates(subset=['region'])
rent = rent.replace([city for city in cities_list['region']], 
                    ['SF/SJ', 'SD', 'JAX', 'CHI', 'NY', 'PHL', 'ATX', 'DFW', 'HOU', 'SA', 'PHX', 'LA'])

# Display data 
rent

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048692414,https://sfbay.craigslist.org/scz/apa/d/capitol...,SF/SJ,https://sfbay.craigslist.org,1650,apartment,325,1,1.0,0,...,0,0,0,no laundry on site,off-street parking,https://images.craigslist.org/00Y0Y_1S9nzXZ8NF...,Lovely Studio With Small patio for your seren...,36.9771,-121.953,ca
1,7049506190,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1450,apartment,700,1,1.0,0,...,0,0,0,laundry on site,off-street parking,https://images.craigslist.org/00G0G_cG4yFLfaAa...,Apartment Details: -1 bedroom 1 bath - Proper...,32.7423,-117.095,ca
2,7043935679,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1700,apartment,700,1,1.0,0,...,0,0,0,laundry on site,NaN,https://images.craigslist.org/00g0g_fjiyS1iI2o...,1 Bedroom with a Great View!!! Minutes to free...,32.7277,-117.165,ca
3,7035359947,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2685,townhouse,1127,2,2.0,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00g0g_dA6FtxqdV9...,The life youâve always imagined is at Levant...,32.7958,-117.071,ca
4,7049978012,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1390,apartment,263,0,1.0,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00v0v_akWNyI1F4d...,THE BARCELONA The perfect pairing of location...,32.7294,-117.162,ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26088,7035391481,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2525,apartment,1127,2,2.0,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00E0E_bs0T3dPEIH...,"Located near Mission Valley, the townhomes at ...",32.7958,-117.071,ca
26089,7049783450,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2593,apartment,1094,2,2.0,1,...,0,0,0,w/d in unit,NaN,https://images.craigslist.org/01414_kiWespdEZL...,To schedule a tour We now book our tour appoin...,32.8274,-117.138,ca
26090,7049426671,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1300,apartment,490,1,1.0,1,...,1,0,1,laundry on site,off-street parking,https://images.craigslist.org/00B0B_bKWtLiBFGY...,Beautiful Turn-Key 1 Bed/1 Bath is located in ...,32.7560,-117.121,ca
26091,7049461332,https://sandiego.craigslist.org/nsd/apa/d/esco...,SD,https://sandiego.craigslist.org,1646,apartment,812,2,1.0,1,...,0,0,0,NaN,NaN,https://images.craigslist.org/00i0i_lgzbYDbzfP...,Bedrooms: 2 Bathrooms: 1 Square Feet: 812 Int...,33.1435,-117.097,ca


In [133]:
## Filter out unreasonable values and missing lat/long
rent = rent.loc[(rent["price"] >= 100) & (rent["price"] <= 20000) & 
                (rent["sqfeet"] >= 200) & (rent["sqfeet"] <= 10000) &
                (rent["beds"] <= 7) & 
                (rent["baths"] <= 7), :]
rent = rent[rent.lat.notnull()]
rent = rent[rent.long.notnull()]
rent

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048692414,https://sfbay.craigslist.org/scz/apa/d/capitol...,SF/SJ,https://sfbay.craigslist.org,1650,apartment,325,1,1.0,0,...,0,0,0,no laundry on site,off-street parking,https://images.craigslist.org/00Y0Y_1S9nzXZ8NF...,Lovely Studio With Small patio for your seren...,36.9771,-121.953,ca
1,7049506190,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1450,apartment,700,1,1.0,0,...,0,0,0,laundry on site,off-street parking,https://images.craigslist.org/00G0G_cG4yFLfaAa...,Apartment Details: -1 bedroom 1 bath - Proper...,32.7423,-117.095,ca
2,7043935679,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1700,apartment,700,1,1.0,0,...,0,0,0,laundry on site,NaN,https://images.craigslist.org/00g0g_fjiyS1iI2o...,1 Bedroom with a Great View!!! Minutes to free...,32.7277,-117.165,ca
3,7035359947,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2685,townhouse,1127,2,2.0,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00g0g_dA6FtxqdV9...,The life youâve always imagined is at Levant...,32.7958,-117.071,ca
4,7049978012,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1390,apartment,263,0,1.0,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00v0v_akWNyI1F4d...,THE BARCELONA The perfect pairing of location...,32.7294,-117.162,ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26088,7035391481,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2525,apartment,1127,2,2.0,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00E0E_bs0T3dPEIH...,"Located near Mission Valley, the townhomes at ...",32.7958,-117.071,ca
26089,7049783450,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2593,apartment,1094,2,2.0,1,...,0,0,0,w/d in unit,NaN,https://images.craigslist.org/01414_kiWespdEZL...,To schedule a tour We now book our tour appoin...,32.8274,-117.138,ca
26090,7049426671,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1300,apartment,490,1,1.0,1,...,1,0,1,laundry on site,off-street parking,https://images.craigslist.org/00B0B_bKWtLiBFGY...,Beautiful Turn-Key 1 Bed/1 Bath is located in ...,32.7560,-117.121,ca
26091,7049461332,https://sandiego.craigslist.org/nsd/apa/d/esco...,SD,https://sandiego.craigslist.org,1646,apartment,812,2,1.0,1,...,0,0,0,NaN,NaN,https://images.craigslist.org/00i0i_lgzbYDbzfP...,Bedrooms: 2 Bathrooms: 1 Square Feet: 812 Int...,33.1435,-117.097,ca


In [151]:
# Group by city and create table of summary stats for 'price'
rent_summary = rent.groupby(["region"])
rent_summary['price'].describe().sort_values(by = 'mean', ascending = False)

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
SF/SJ,2307.0,3074.712614,1460.015907,1000.0,2250.0,2750.0,3500.0,20000.0
NY,723.0,2643.741355,1108.468553,790.0,1900.0,2400.0,3099.0,13125.0
LA,2232.0,2509.583333,1130.463921,510.0,1800.0,2295.0,2910.0,18500.0
SD,2346.0,2254.057545,847.498350,100.0,1746.0,2056.0,2550.0,11995.0
CHI,1483.0,1723.163857,847.958760,450.0,1196.0,1500.0,2000.0,10000.0
PHL,1833.0,1620.361702,679.338637,149.0,1171.0,1495.0,1925.0,5795.0
ATX,2418.0,1379.616212,532.091338,455.0,1053.0,1275.0,1570.0,12975.0
DFW,2265.0,1294.348786,479.278222,116.0,976.0,1218.0,1520.0,6800.0
PHX,2036.0,1208.961198,397.668446,209.0,975.0,1180.0,1370.0,5500.0


In [135]:
# Group by city and create table of summary stats for 'sqfeet'
rent_summary['sqfeet'].describe().sort_values(by = 'mean', ascending = False)

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
CHI,1483.0,1033.334457,481.480698,250.0,710.0,924.0,1200.00,5335.0
HOU,1700.0,1018.604706,508.595897,200.0,700.0,900.0,1140.75,4050.0
PHL,1833.0,964.278232,385.265936,300.0,700.0,900.0,1120.00,3072.0
SF/SJ,2307.0,962.788903,507.728955,200.0,700.0,852.0,1096.00,6500.0
SD,2346.0,958.153026,395.661112,200.0,700.0,900.0,1120.75,5000.0
PHX,2036.0,957.498527,427.230769,240.0,720.0,900.0,1100.00,7290.0
JAX,4211.0,955.769651,307.201847,288.0,753.0,920.0,1105.00,4005.0
DFW,2265.0,953.381457,336.139928,247.0,716.0,907.0,1106.00,4455.0
SA,2041.0,951.847134,398.443805,260.0,675.0,915.0,1100.00,3858.0


In [161]:
# Group by city and create table of summary stats for 'price_per_sqfeet'
price = rent_summary['price'].sum()
sqfeet = rent_summary['sqfeet'].sum()
price_per_sqfeet = pd.DataFrame({"price": price,
                                 "sq_feet": sqfeet,
                                 "price_per_sqfeet": price / sqfeet})

price_per_sqfeet_summary = price_per_sqfeet.groupby(["region"])
price_per_sqfeet_summary['price_per_sqfeet'].describe().sort_values(by = 'mean', ascending = False).iloc[:, [0,1]]

,count,mean
region,,
SF/SJ,1.0,3.193548
NY,1.0,2.904914
LA,1.0,2.728347
SD,1.0,2.352503
PHL,1.0,1.680388
CHI,1.0,1.667576
ATX,1.0,1.460672
DFW,1.0,1.357640
PHX,1.0,1.262625


In [136]:
# Group by city and create table of summary stats for 'beds'
rent_summary['beds'].describe().sort_values(by = 'mean', ascending = False)

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
JAX,4211.0,1.853716,0.731362,0.0,1.0,2.0,2.0,6.0
PHX,2036.0,1.774558,0.927209,0.0,1.0,2.0,2.0,6.0
SA,2041.0,1.761391,0.912011,0.0,1.0,2.0,2.0,5.0
CHI,1483.0,1.749831,0.939531,0.0,1.0,2.0,2.0,5.0
SD,2346.0,1.738278,0.883708,0.0,1.0,2.0,2.0,5.0
PHL,1833.0,1.695035,0.934692,0.0,1.0,2.0,2.0,6.0
NY,723.0,1.686030,1.045852,0.0,1.0,2.0,2.0,5.0
HOU,1700.0,1.684706,0.838663,0.0,1.0,2.0,2.0,5.0
SF/SJ,2307.0,1.666233,0.937093,0.0,1.0,2.0,2.0,7.0


In [137]:
# Group by city and create table of summary stats for 'baths'
rent_summary['baths'].describe().sort_values(by = 'mean', ascending = False)

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
SD,2346.0,1.524084,0.596861,0.0,1.0,1.0,2.0,4.5
PHX,2036.0,1.518418,0.565841,0.0,1.0,1.5,2.0,4.0
SA,2041.0,1.504900,0.648509,0.0,1.0,1.0,2.0,5.0
HOU,1700.0,1.500294,0.612192,0.0,1.0,1.0,2.0,4.0
ATX,2418.0,1.473325,0.610395,0.0,1.0,1.0,2.0,4.0
JAX,4211.0,1.429589,0.525337,0.0,1.0,1.0,2.0,4.0
SF/SJ,2307.0,1.429345,0.622811,0.0,1.0,1.0,2.0,5.5
LA,2232.0,1.426075,0.600613,0.0,1.0,1.0,2.0,6.5
DFW,2265.0,1.415894,0.588549,0.0,1.0,1.0,2.0,4.0


In [144]:
# Group by city and create table of % 'dogs_allowed'
rent_summary['dogs_allowed'].describe().sort_values(by = 'mean', ascending = False).iloc[:, [0,1]]

,count,mean
region,,
ATX,2418.0,0.777916
DFW,2265.0,0.750993
PHX,2036.0,0.724951
JAX,4211.0,0.706958
HOU,1700.0,0.646471
SA,2041.0,0.643312
SD,2346.0,0.635550
LA,2232.0,0.622760
CHI,1483.0,0.621713


In [145]:
# Group by city and create table of % 'smoking_allowed'
rent_summary['smoking_allowed'].describe().sort_values(by = 'mean', ascending = False).iloc[:, [0,1]]

,count,mean
region,,
ATX,2418.0,0.889992
PHX,2036.0,0.887033
SA,2041.0,0.880941
HOU,1700.0,0.870000
PHL,1833.0,0.849973
JAX,4211.0,0.844930
DFW,2265.0,0.844592
NY,723.0,0.799447
LA,2232.0,0.689964


In [146]:
# Group by city and create table of % 'electric_vehicle_charge'
rent_summary['electric_vehicle_charge'].describe().sort_values(by = 'mean', ascending = False).iloc[:, [0,1]]

,count,mean
region,,
SF/SJ,2307.0,0.100997
LA,2232.0,0.094982
SD,2346.0,0.071185
HOU,1700.0,0.018235
PHL,1833.0,0.014730
NY,723.0,0.011065
CHI,1483.0,0.010789
ATX,2418.0,0.009098
PHX,2036.0,0.003929


In [99]:
## Import income data
# Source: https://www.bea.gov/data/income-saving/personal-income-county-metro-and-other-areas ('Metropolitian Area Table')
income = "Resources/income_data.xlsx"
income = pd.read_excel(income)

# Select relevant rows and columns
income = income.iloc[7:391, [0,2]]

# Rename columns
income.rename(columns = {'Table 2. Per Capita Personal Income, by Metropolitan Area, 2019–2021': "region", 
                         'Unnamed: 2': "Per_Capita_Income_2020"}, inplace = True)

# Filter for major cities
city_list = ['San Francisco-Oakland-Berkeley, CA', 'San Jose-Sunnyvale-Santa Clara, CA', 'San Diego-Chula Vista-Carlsbad, CA', 
             'Jacksonville, FL', 'Chicago-Naperville-Elgin, IL-IN-WI', 'New York-Newark-Jersey City, NY-NJ-PA', 
             'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', 'Austin-Round Rock-Georgetown, TX', 'Dallas-Fort Worth-Arlington, TX', 
             'Houston-The Woodlands-Sugar Land, TX', 'San Antonio-New Braunfels, TX', 'Phoenix-Mesa-Chandler, AZ', 
             'Los Angeles-Long Beach-Anaheim, CA']
income_filter = income.loc[income['region'].isin(city_list)]

# Standardize city names
income = income_filter.replace([city for city in income_filter['region']], 
                               ['ATX', 'CHI', 'DFW', 'HOU', 'JAX', 'LA', 'NY', 'PHL', 'PHX', 'SA', 'SD', 'SF/SJ', 'SF/SJ'])

# Group by cities in order to combine 'San Francisco & San Jose'
income = income.groupby(['region']).mean()

# Display Data
income

,Per_Capita_Income_2020
region,
ATX,64916.0
CHI,66474.0
DFW,61824.0
HOU,61133.0
JAX,55158.0
LA,70280.0
NY,78727.0
PHL,68200.0
PHX,54907.0


In [100]:
## Merge Rent and Income data
data = pd.merge(rent, income, how="left", on = "region")
data

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state,Per_Capita_Income_2020
0,7048692414,https://sfbay.craigslist.org/scz/apa/d/capitol...,SF/SJ,https://sfbay.craigslist.org,1650,apartment,325,1,1.0,0,...,0,0,no laundry on site,off-street parking,https://images.craigslist.org/00Y0Y_1S9nzXZ8NF...,Lovely Studio With Small patio for your seren...,36.9771,-121.953,ca,115573.5
1,7049506190,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1450,apartment,700,1,1.0,0,...,0,0,laundry on site,off-street parking,https://images.craigslist.org/00G0G_cG4yFLfaAa...,Apartment Details: -1 bedroom 1 bath - Proper...,32.7423,-117.095,ca,67830.0
2,7043935679,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1700,apartment,700,1,1.0,0,...,0,0,laundry on site,NaN,https://images.craigslist.org/00g0g_fjiyS1iI2o...,1 Bedroom with a Great View!!! Minutes to free...,32.7277,-117.165,ca,67830.0
3,7035359947,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2685,townhouse,1127,2,2.0,1,...,0,0,w/d in unit,attached garage,https://images.craigslist.org/00g0g_dA6FtxqdV9...,The life youâve always imagined is at Levant...,32.7958,-117.071,ca,67830.0
4,7049978012,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1390,apartment,263,0,1.0,1,...,0,0,laundry on site,street parking,https://images.craigslist.org/00v0v_akWNyI1F4d...,THE BARCELONA The perfect pairing of location...,32.7294,-117.162,ca,67830.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26088,7035391481,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2525,apartment,1127,2,2.0,1,...,0,0,w/d in unit,attached garage,https://images.craigslist.org/00E0E_bs0T3dPEIH...,"Located near Mission Valley, the townhomes at ...",32.7958,-117.071,ca,67830.0
26089,7049783450,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,2593,apartment,1094,2,2.0,1,...,0,0,w/d in unit,NaN,https://images.craigslist.org/01414_kiWespdEZL...,To schedule a tour We now book our tour appoin...,32.8274,-117.138,ca,67830.0
26090,7049426671,https://sandiego.craigslist.org/csd/apa/d/san-...,SD,https://sandiego.craigslist.org,1300,apartment,490,1,1.0,1,...,0,1,laundry on site,off-street parking,https://images.craigslist.org/00B0B_bKWtLiBFGY...,Beautiful Turn-Key 1 Bed/1 Bath is located in ...,32.7560,-117.121,ca,67830.0
26091,7049461332,https://sandiego.craigslist.org/nsd/apa/d/esco...,SD,https://sandiego.craigslist.org,1646,apartment,812,2,1.0,1,...,0,0,NaN,NaN,https://images.craigslist.org/00i0i_lgzbYDbzfP...,Bedrooms: 2 Bathrooms: 1 Square Feet: 812 Int...,33.1435,-117.097,ca,67830.0


In [118]:
## Census File to Load
# Original data source: https://worldpopulationreview.com/us-cities
# Updated for 2020 Census, includes 2023 population

# Read file and display
census = "Resources/census_data.csv"
census = pd.read_csv(census)

# Filter for major cities
city_list = ['San Francisco', 'San Jose', 'San Diego', 'Jacksonville', 'Chicago', 'New York City', 'Philadelphia', 'Austin', 
             'Dallas', 'Fort Worth', 'Houston', 'San Antonio', 'Phoenix', 'Los Angeles']
census_filter = census.loc[census['name'].isin(city_list)]

# Standardize city names
census = census_filter.replace([city for city in census_filter['name']], 
                               ['NY', 'LA', 'CHI', 'HOU', 'PHX', 'PHL', 'SA', 'SD', 'DFW', 'SF/SJ', 'ATX', 'JAX', 'DFW', 'SF/SJ'])

# Group by cities in order to combine 'Dallas & Forth Worth' and 'San Francisco & San Jose'
census = census.groupby(['name']).sum()
census

,pop2023,pop2022,pop2020,pop2010,growth,geoid,ansicode,funcstat,aland,awater,aland_sqmi,awater_sqmi,intptlat,intptlong,metroId,rank,density
name,,,,,,,,,,,,,,,,,
ATX,1013293,996147,961855,790390,0.01721,4805000,2409761,0,828638000,17024700,319.939,6.573,30.3012,-97.7527,12420.0,11,3167
CHI,2761625,2756546,2746388,2695598,0.00184,1714000,428803,0,588882000,17621500,227.369,6.804,41.8376,-87.6818,16980.0,3,12146
DFW,2308575,2280148,2223294,1939022,0.02666,9646000,4820819,0,1774970000,137481300,685.320,53.082,65.5748,-194.1132,38200.0,22,6746
HOU,2366119,2345606,2304580,2099451,0.00875,4835000,2410796,0,1658090000,81248100,640.194,31.370,29.7857,-95.3888,26420.0,4,3696
JAX,987960,975177,949611,821784,0.01311,1235000,2404783,0,1935930000,329458000,747.467,127.204,30.3369,-81.6616,27260.0,12,1322
LA,3930586,3919973,3898747,3792621,0.00271,644000,2410877,0,1214590000,87468500,468.956,33.772,34.0194,-118.4110,31080.0,2,8382
NY,8992908,8930002,8804190,8175133,0.00704,3651000,2395220,0,777983000,434642000,300.381,167.816,40.6635,-73.9387,35620.0,1,29938
PHL,1627134,1619355,1603797,1526006,0.00480,4260000,1215531,0,347782000,21806100,134.279,8.419,40.0094,-75.1333,37980.0,6,12118
PHX,1656892,1640641,1608139,1445632,0.00991,455000,2411414,0,1340770000,3236740,517.673,1.250,33.5722,-112.0900,38060.0,5,3201


In [119]:
## Merge Datasets on city
data = pd.merge(data, census, how="left", left_on = "region", right_on = "name")

# Delete unused columns
data = data.drop(columns = ['url', 'region_url', 'image_url', 'pop2023', 'pop2022', 'growth', 'geoid', 'ansicode', 'funcstat', 'intptlat', 
                            'intptlong', 'metroId', 'rank', 'density'])

# Display data
data

,id,region,price,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,...,lat,long,state,Per_Capita_Income_2020,pop2020,pop2010,aland,awater,aland_sqmi,awater_sqmi
0,7048692414,SF/SJ,1650,apartment,325,1,1.0,0,0,0,...,36.9771,-121.953,ca,115573.5,1887205,1751177,581999000,487244850,224.711,188.126
1,7049506190,SD,1450,apartment,700,1,1.0,0,0,0,...,32.7423,-117.095,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540
2,7043935679,SD,1700,apartment,700,1,1.0,0,0,1,...,32.7277,-117.165,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540
3,7035359947,SD,2685,townhouse,1127,2,2.0,1,1,1,...,32.7958,-117.071,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540
4,7049978012,SD,1390,apartment,263,0,1.0,1,1,0,...,32.7294,-117.162,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26088,7035391481,SD,2525,apartment,1127,2,2.0,1,1,1,...,32.7958,-117.071,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540
26089,7049783450,SD,2593,apartment,1094,2,2.0,1,1,1,...,32.8274,-117.138,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540
26090,7049426671,SD,1300,apartment,490,1,1.0,1,1,1,...,32.7560,-117.121,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540
26091,7049461332,SD,1646,apartment,812,2,1.0,1,1,1,...,33.1435,-117.097,ca,67830.0,1386932,1307402,844018000,120537000,325.877,46.540


In [121]:
## Add calculated columns
data['Density'] = data['pop2020'] / data['aland_sqmi']
data['Growth'] = (data['pop2020'] / data['pop2010']) - 1

In [122]:
## Write to csv
data.to_csv('Resources/final_data.csv')